In [4]:
import pandas as pd
import numpy as np

np.random.seed(42)

# 1) 读取数据
df = pd.read_excel("PT_Dataset.xlsx").copy()
n = len(df)

# 2) 资格与继续教育（与行业规则对齐：持证者更可能持续进修）
qual_opts = ['ACE','NASM','ISSA','None']
qual_p    = [0.28, 0.30, 0.22, 0.20]   # 三大主流+无证，允许“无证”存在以产生区分
df['Qualifications'] = np.random.choice(qual_opts, size=n, p=qual_p)

exp = df['years_experience'].fillna(3).clip(0, 30)
has_cert = (df['Qualifications']!='None').astype(int)

# 简化的继续教育等级(0-3)：与经验/是否持证相关
cont_base = (exp/5).clip(0,6)*0.6 + has_cert*1.2
df['Continuing_Education'] = np.random.poisson(lam=np.clip(cont_base, 0.2, 3.2)).clip(0,3)

# 3) Online Presence（平台组合）
presence_choices = ['Website','Instagram','Facebook',
                    'Website+Instagram','Instagram+Facebook',
                    'Website+Facebook','All','None']
presence_probs   = [0.18,0.22,0.10,0.18,0.12,0.08,0.07,0.05]
df['Online_Presence'] = np.random.choice(presence_choices, size=n, p=presence_probs)

def presence_count(s):
    if s == 'All': return 3
    if s == 'None': return 0
    return s.count('+') + 1

pres_num = df['Online_Presence'].apply(presence_count)

# 4) 基础行为变量
likes   = df['total_likes'].fillna(0).astype(float)
videos  = df['videos_count'].fillna(0).astype(float)
rating  = df['athlete_rating'].fillna(4.2).clip(1,5)

# 5) 线索/转化/活跃客户（用于留存与供需）
content_factor = np.log1p(videos).clip(0,6)
lead_score = 0.8*pres_num + 0.6*content_factor + 0.7*(rating-3.5)
monthly_leads = np.random.poisson(lam=np.clip(2 + lead_score, 0.5, 20))

conv_rate = (0.22 + 0.03*(rating-4.0) + 0.02*has_cert + 0.01*df['Continuing_Education']).clip(0.08, 0.55)
active_clients = np.random.binomial(n=np.maximum(monthly_leads,1), p=conv_rate) + (exp/7).astype(int)

# 6) 留存率（行业先验：月留存~0.90–0.94，质量加成）
base_month_ret = np.random.normal(0.92, 0.015, size=n).clip(0.86, 0.96)
quality_bump = 0.01*has_cert + 0.005*df['Continuing_Education'] + 0.01*(rating-4.0)
df['Retention_Rate'] = (base_month_ret + quality_bump).clip(0.85, 0.98).round(3)

# 7) Testimonials ——“口碑互动指标”版本
# 定义：按行业基准，将“互动率(ER)”映射到总点赞与作品数上，外加多平台与高评分加成
# 7.1 互动基础：likes × (ER抽样) + videos × 额外互动常数
er = np.random.uniform(0.010, 0.030, size=n)    # 1%~3% 的互动率区间（见参考）
extra_per_video = np.random.uniform(0.5, 1.2, size=n)  # 每个视频带来的额外互动（评论/分享等）

base_interactions = likes * er + videos * extra_per_video

# 7.2 多平台与评分加成
platform_factor = 1 + 0.05*(pres_num-1)        # 每多一个平台约+5%，三平台上限~+10%
rating_factor   = np.where(rating>=4.5, 1.08, 1.00)

# 7.3 随机波动与范围裁剪
noise = np.random.uniform(0.85, 1.15, size=n)
testi_engage = base_interactions * platform_factor * rating_factor * noise

# 合理上下限：至少10条，顶到5000条（防止极端值）
df['Testimonials'] = np.clip(testi_engage, 10, 8000).astype(int)

# 8) 档案完整度（字段完备+内容强度）
comp_raw = (
    has_cert +
    (df['Continuing_Education']>0).astype(int) +
    (pres_num>0).astype(int) +
    (df['Testimonials']>=50).astype(int) +
    (videos>=30).astype(int)
)
df['Profile_Completeness'] = np.clip(0.35 + 0.12*comp_raw + np.random.normal(0,0.03,n), 0.30, 0.99).round(2)

# 9) 定价（$/h）：经验+认证+评分+供需
cert_bonus = df['Qualifications'].map({'ACE':6,'NASM':7,'ISSA':5,'None':0}).fillna(0)
busy_factor = np.clip(active_clients - 8, -6, 12)  # 客户越多越忙，价格更硬
price = (30 + 1.8*exp + cert_bonus + 4*(rating-4.0) + 0.9*busy_factor +
         np.random.normal(0,6,n))
df['Pricing'] = np.clip(price, 30, 125).round(2)  # 行业范围锚定（见参考）

# 10) 可用时长
hours = np.random.randint(12, 36, size=n)
hours[:int(0.1*n)] = np.random.randint(38, 55, size=int(0.1*n))  # 10% 高工时
np.random.shuffle(hours)
df['Available_Hours_Per_Week'] = hours

# 11) 保存
df.to_excel("Full_data.xlsx", index=False)
print("✅ 口碑互动版 数据增强完成，已保存为 Full_data.xlsx")


✅ 口碑互动版 数据增强完成，已保存为 Full_data.xlsx
